In [1]:
!pip install faster-whisper flask transformers torch gtts pydub pyngrok ffmpeg-python
!apt-get install -y ffmpeg


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.

In [2]:
!ngrok config add-authtoken 2voIhy3pZ30RBf6X0FTTX9YtLpo_3bsnLV7UDJuvSnp15dG1X

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
from pyngrok import ngrok

# Kill any existing ngrok processes
!pkill ngrok

# Now start your new tunnel
public_url = ngrok.connect(5000)
print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://c587-35-227-9-134.ngrok-free.app" -> "http://localhost:5000"


In [4]:
# End-to-End Funny Audio Rewriter Agent (Using DeepSeek LLM)

# Required installs:
# pip install faster-whisper flask transformers torch gtts pydub

import os
from flask import Flask, request, jsonify, send_file
from gtts import gTTS
from pydub import AudioSegment
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from faster_whisper import WhisperModel
from threading import Thread

app = Flask(__name__)

# Load Whisper model for transcription
whisper_model = WhisperModel("base", compute_type="float32")

# Load DeepSeek open-source LLM
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-1.3b-base")
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-1.3b-base")
text_gen = pipeline("text-generation", model=model, tokenizer=tokenizer)

@app.route("/process-audio", methods=["POST"])
def process_audio():
    if 'audio' not in request.files:
        return jsonify({"error": "No audio file provided"}), 400

    audio_file = request.files['audio']
    audio_path = "11.wav"
    audio_file.save(audio_path)

    # Step 1: Transcribe audio to text
    segments, _ = whisper_model.transcribe(audio_path)
    original_text = " ".join([segment.text for segment in segments])

    # Step 2: Generate expressive version (without changing meaning)
    funny_text = make_it_funny(original_text)

    # Step 3: Convert expressive text to speech
    tts = gTTS(funny_text)
    tts_path = "funny_output.mp3"
    tts.save(tts_path)
    AudioSegment.from_mp3(tts_path).export("funny_output.wav", format="wav")

    return send_file("funny_output.wav", mimetype="audio/wav")

def make_it_funny(text):
    prompt = f"Add dramatic pauses, fumbles, and exaggerated expressions to this spoken sentence, but do not change its meaning or wording: \"{text}\""
    output = text_gen(prompt, max_length=100, do_sample=True, top_p=0.95, temperature=0.8)[0]['generated_text']
    # Remove the prompt from the output if needed
    processed = output.replace(prompt, "").strip()
    return processed

# Launch Flask in background
Thread(target=lambda: app.run(port=5000)).start()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


config.json:   0%|          | 0.00/2.31k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/145M [00:00<?, ?B/s]

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

Device set to use cpu


 * Serving Flask app '__main__'
 * Debug mode: off


In [5]:
from google.colab import files
uploaded = files.upload()


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Saving 11.wav to 11.wav


In [6]:
import requests

file_name = list(uploaded.keys())[0]
with open(file_name, 'rb') as f:
    # Get the public URL as a string using public_url.public_url
    response = requests.post(public_url.public_url + "/process-audio", files={"audio": f})

with open("funny_result.wav", "wb") as out:
    out.write(response.content)

from IPython.display import Audio
Audio("funny_result.wav")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
INFO:werkzeug:127.0.0.1 - - [16/Apr/2025 12:47:11] "POST /process-audio HTTP/1.1" 200 -
